# Laboratorio 6
## Data Science
Javier Alejandro Ovalle Chiquín, 22103  
José Ángel Morales Farfán, 22689  
Ricardo Josué Morales Contreras, 22289

In [10]:
import pandas as pd
import json, re
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix



In [ ]:
# ---------- Lectura robusta ----------
def read_text_robust(path):
    for enc in ("utf-8-sig", "utf-16", "latin-1", "utf-8"):
        try:
            with open(path, "r", encoding=enc) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
    with open(path, "rb") as f:
        return f.read().decode("utf-8", "ignore")

# ---------- Parser JSONL tolerante ----------
def parse_jsonl(content: str):
    recs = []
    for line in content.strip().splitlines():
        s = line.strip()
        if not s:
            continue
        # quitar coma final y quedarnos con {...}
        if s.endswith(","):
            s = s[:-1].rstrip()
        if "{" in s and "}" in s:
            s = s[s.find("{"): s.rfind("}") + 1]
        try:
            obj = json.loads(s)
            if isinstance(obj, dict):
                recs.append(obj)
        except Exception:
            # si una línea falla, la saltamos
            continue
    return recs

# ---------- Normalización a columnas ----------
def flatten_tweet(obj: dict) -> dict:
    user = obj.get("user") or {}
    if isinstance(user, dict):
        username    = user.get("username") or user.get("screen_name")
        displayname = user.get("displayname") or user.get("name")
    else:
        username, displayname = (None, None)

    # mentionedUsers puede venir como lista de dicts
    musers = []
    if isinstance(obj.get("mentionedUsers"), list):
        for m in obj["mentionedUsers"]:
            if isinstance(m, dict):
                u = m.get("username") or m.get("screen_name")
                if u: musers.append(u)

    # hashtags pueden venir como lista de strings
    hashtags = obj.get("hashtags") if isinstance(obj.get("hashtags"), list) else []

    # links pueden venir como lista de dicts con 'url'
    links = []
    if isinstance(obj.get("links"), list):
        for l in obj["links"]:
            if isinstance(l, dict):
                u = l.get("url") or l.get("expanded_url")
                if u: links.append(u)

    in_reply_to_user = None
    if isinstance(obj.get("inReplyToUser"), dict):
        in_reply_to_user = obj["inReplyToUser"].get("username") or obj["inReplyToUser"].get("screen_name")

    return {
        "tweet_id": str(obj.get("id") or obj.get("id_str") or ""),
        "url": obj.get("url"),
        "date": pd.to_datetime(obj.get("date"), utc=True, errors="coerce"),
        "username": username,
        "displayname": displayname,
        "text": obj.get("rawContent") or obj.get("full_text") or obj.get("text"),
        "reply_count": obj.get("replyCount") or obj.get("reply_count"),
        "retweet_count": obj.get("retweetCount") or obj.get("retweet_count"),
        "like_count": obj.get("likeCount") or obj.get("favorite_count"),
        "quote_count": obj.get("quoteCount") or obj.get("quote_count"),
        "mentioned_users": musers,
        "hashtags": hashtags,
        "links": links,
        "is_retweet": obj.get("retweetedTweet") is not None,
        "is_quote": obj.get("quotedTweet") is not None,
        "in_reply_to_user": in_reply_to_user,
        "lang": obj.get("lang"),
        "sourceLabel": obj.get("sourceLabel") or obj.get("source"),
    }

def load_tweets(path: str) -> pd.DataFrame:
    content = read_text_robust(path)
    recs = parse_jsonl(content)
    rows = [flatten_tweet(r) for r in recs]
    df = pd.DataFrame(rows)

    # columnas ordenadas
    order = ["tweet_id","date","username","displayname","text",
             "reply_count","retweet_count","like_count","quote_count",
             "is_retweet","is_quote","in_reply_to_user",
             "mentioned_users","hashtags","links","url","lang","sourceLabel"]
    df = df[[c for c in order if c in df.columns]].copy()

    # opcional: fecha local
    if "date" in df and pd.api.types.is_datetime64_any_dtype(df["date"]):
        try:
            df["date_local"] = df["date"].dt.tz_convert("America/Guatemala")
        except Exception:
            pass
    return df

# --------- Usa el loader con tus archivos ----------
df_trafico  = load_tweets("traficogt.txt")
# Si también tienes 'tioberny.txt', descomenta:
# df_tioberny = load_tweets("tioberny.txt")

print("traficogt:", df_trafico.shape)
print(df_trafico.head(5))


traficogt: (5604, 19)
              tweet_id                      date         username  \
0  1834236045598056867 2024-09-12 14:22:06+00:00        traficogt   
1  1834029142565658846 2024-09-12 00:39:56+00:00     monymmorales   
2  1834039491826180424 2024-09-12 01:21:04+00:00  animaldgalaccia   
3  1833963729136091179 2024-09-11 20:20:01+00:00   EstacionDobleA   
4  1833665391698092330 2024-09-11 00:34:31+00:00       CubReserva   

                 displayname  \
0                  traficoGT   
1                       Mony   
2           Jairo De La Nada   
3           Estación Doble A   
4  CUB Reserva Kanajuyu Z 16   

                                                text  reply_count  \
0  Es comprensible la resolución... El ruso sabe ...          NaN   
1  La corrupción de la @CC_Guatemala\nes descarad...          NaN   
2  @PNCdeGuatemala @mingobguate @FJimenezmingob @...          NaN   
3  @amilcarmontejo @AztecaNoticiaGT @BancadaSemil...          NaN   
4  @soy_502 @AztecaNotici

#### 3) Limpieza de los datos   

#### 3.1) Quitar caracteres especiales 

In [ ]:
# --- Expresiones regulares ---
URL_RE      = re.compile(r"https?://\S+")
MENTION_RE  = re.compile(r"@\w{1,20}")
HASHTAG_RE  = re.compile(r"#\w+")
EMOJI_RE    = re.compile(
    "[" 
    "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
    "\U0001F600-\U0001F64F"  # emoticones
    "\U0001F680-\U0001F6FF"  # transporte
    "\U0001F700-\U0001F77F"
    "\U0001F900-\U0001F9FF"
    "\U00002600-\U000026FF"  # misceláneos
    "\U00002700-\U000027BF"
    "]+", flags=re.UNICODE
)
NONWORD_RE  = re.compile(r"[^a-záéíóúñü\s]")  # quita números y símbolos
MULTISPACE  = re.compile(r"\s+")

# Stopwords en español (artículos, preposiciones, conjunciones)
STOPWORDS_ES = set("""
a al algo algunas alguno algunos ante antes como con contra cual cuales cuando de del desde donde
el la los las un una unos unas
e en entre era erais éramos eran es esa esas ese eso esos esta estas este estos
haber había habían habrá hay hasta incluso mas más me mi mis muy nada ni no nos nosotros
o os otra otras otro otros para pero poco por porque que se sea sean ser será si sido sin sobre soy
su sus tal también tampoco tan tanto te tener tenía tenían tengo ti tuvo usted ustedes ya yo
""".split())

# --- Función de limpieza ---
def clean_text_31(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()                         # minúsculas
    s = URL_RE.sub(" ", s)                # quitar urls
    s = EMOJI_RE.sub(" ", s)              # quitar emoticones
    s = MENTION_RE.sub(" ", s)            # quitar menciones @
    s = HASHTAG_RE.sub(" ", s)            # quitar hashtags #
    s = NONWORD_RE.sub(" ", s)            # quitar signos y números
    s = MULTISPACE.sub(" ", s).strip()    # espacios múltiples -> uno solo
    # eliminar stopwords
    tokens = [t for t in s.split() if t not in STOPWORDS_ES]
    return " ".join(tokens)

# --- Aplicar sobre tu DataFrame ---
df_trafico["text_clean"] = df_trafico["text"].apply(clean_text_31)

# Vista previa
print(df_trafico[["text","text_clean"]].head(10))


                                                text  \
0  Es comprensible la resolución... El ruso sabe ...   
1  La corrupción de la @CC_Guatemala\nes descarad...   
2  @PNCdeGuatemala @mingobguate @FJimenezmingob @...   
3  @amilcarmontejo @AztecaNoticiaGT @BancadaSemil...   
4  @soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...   
5  @amilcarmontejo @PMTMuniGuate @Noti7Guatemala ...   
6  Favor compartir  \nEl vive el zona 7 Bethania ...   
7  @traficogt @_ojoconmipisto @soy_502 @AztecaNot...   
8  @piero_coen @FJimenezmingob @traficogt @mingob...   
9  @erwin_fern84019 @piero_coen @FJimenezmingob @...   

                                          text_clean  
0  comprensible resolución ruso sabe engrasar maq...  
1  corrupción descarada falsificación documentos ...  
2                                                     
3                                                     
4  urgente zona deterioro tala inmoderada tráfico...  
5  avenidas y calles avenida calle luz semáforo y... 

#### 3.2) Extraer menciones, respuestas y rt

In [7]:
# --- Patrones útiles ---
MENTION_RE = re.compile(r'@([A-Za-z0-9_]{1,20})')
RETWEET_RE = re.compile(r'^\s*rt\s+@([A-Za-z0-9_]{1,20})', re.IGNORECASE)
REPLY_HEAD_RE = re.compile(r'^\s*@([A-Za-z0-9_]{1,20})')  # a veces las respuestas empiezan con @user

def safe_list(x):
    return x if isinstance(x, list) else ([] if pd.isna(x) or x is None else list(x))

def extract_relations(df):
    df = df.copy()

    # 1) MENCIONES: usar campo estructurado si existe; si no, extraer del texto
    if "mentioned_users" in df.columns:
        mentions_col = df["mentioned_users"].apply(safe_list)
    else:
        mentions_col = df["text"].fillna("").apply(lambda s: list(set(MENTION_RE.findall(s))))

    df["mentions_extracted"] = mentions_col
    df["n_mentions"] = df["mentions_extracted"].apply(len)

    # 2) RESPUESTAS: usar campo estructurado si existe; si no, inferir del inicio del texto
    if "in_reply_to_user" in df.columns:
        reply_to = df["in_reply_to_user"]
    else:
        reply_to = df["text"].fillna("").apply(lambda s: (REPLY_HEAD_RE.match(s) or [None, None])[1])

    df["reply_to_user"] = reply_to
    df["is_reply"] = df["reply_to_user"].notna()

    # 3) RETWEETS: usar bandera/campo estructurado si existe; si no, inferir con "RT @user"
    if "is_retweet" in df.columns:
        is_rt = df["is_retweet"].fillna(False).astype(bool)
    else:
        is_rt = df["text"].fillna("").str.contains(r'^\s*rt\s+@', case=False, na=False)

    df["is_retweet_flag"] = is_rt

    # Usuario retuiteado: preferir estructura si existiera; si no, regex
    if "retweeted_user" in df.columns:
        rt_user = df["retweeted_user"]
    else:
        rt_user = df["text"].fillna("").apply(lambda s: (RETWEET_RE.match(s) or [None, None])[1])

    df["retweet_user"] = rt_user

    # 4) Construir aristas dirigidas (src -> dst) por tipo de interacción
    edges = []

    for _, r in df.iterrows():
        src = r.get("username")
        tid = r.get("tweet_id")
        dt  = r.get("date")

        # mention edges
        for dst in r.get("mentions_extracted", []):
            if src and dst and src.lower() != dst.lower():
                edges.append({"src": src.lower(), "dst": dst.lower(), "type": "mention", "tweet_id": tid, "date": dt})

        # reply edge
        dst = r.get("reply_to_user")
        if src and dst and src.lower() != str(dst).lower():
            edges.append({"src": src.lower(), "dst": str(dst).lower(), "type": "reply", "tweet_id": tid, "date": dt})

        # retweet edge
        if r.get("is_retweet_flag"):
            dst = r.get("retweet_user")
            if src and dst and src.lower() != str(dst).lower():
                edges.append({"src": src.lower(), "dst": str(dst).lower(), "type": "retweet", "tweet_id": tid, "date": dt})

    E = pd.DataFrame(edges)
    # Limpieza básica de aristas
    if not E.empty:
        E = E.dropna(subset=["src","dst"])
        E = E[E["src"] != E["dst"]].reset_index(drop=True)

    return df, E

# === Ejecutar 3.2 sobre tu df ===
df_trafico_rel, edges_trafico = extract_relations(df_trafico)

print("Tweets con relaciones:", df_trafico_rel.shape)
print("Aristas construidas:", edges_trafico.shape)
print(edges_trafico.head(10))


Tweets con relaciones: (5604, 26)
Aristas construidas: (15009, 5)
               src              dst     type             tweet_id  \
0     monymmorales     cc_guatemala  mention  1834029142565658846   
1  animaldgalaccia   pncdeguatemala  mention  1834039491826180424   
2  animaldgalaccia      mingobguate  mention  1834039491826180424   
3  animaldgalaccia   fjimenezmingob  mention  1834039491826180424   
4  animaldgalaccia     diegoedeleon  mention  1834039491826180424   
5  animaldgalaccia   amilcarmontejo  mention  1834039491826180424   
6  animaldgalaccia        traficogt  mention  1834039491826180424   
7  animaldgalaccia   pncdeguatemala    reply  1834039491826180424   
8   estaciondoblea   amilcarmontejo  mention  1833963729136091179   
9   estaciondoblea  aztecanoticiagt  mention  1833963729136091179   

                       date  
0 2024-09-12 00:39:56+00:00  
1 2024-09-12 01:21:04+00:00  
2 2024-09-12 01:21:04+00:00  
3 2024-09-12 01:21:04+00:00  
4 2024-09-12 01:21:04+00

In [8]:
# 1) Distribución por tipo de relación
print(edges_trafico["type"].value_counts())

# 2) ¿Hay nodos vacíos o self-loops?
print("src vacíos:", edges_trafico["src"].isna().sum(),
      "dst vacíos:", edges_trafico["dst"].isna().sum())
print("self-loops:", (edges_trafico["src"] == edges_trafico["dst"]).sum())

# 3) ¿Fechas razonables y usuarios únicos?
print(edges_trafico["date"].min(), "→", edges_trafico["date"].max())
print("Usuarios únicos (src ∪ dst):",
      pd.Index(edges_trafico["src"]).union(pd.Index(edges_trafico["dst"])).nunique())


type
mention    10928
reply       4081
Name: count, dtype: int64
src vacíos: 0 dst vacíos: 0
self-loops: 0
2023-08-14 04:35:46+00:00 → 2024-09-12 01:21:04+00:00
Usuarios únicos (src ∪ dst): 2720


#### 3.3) Procesar datos duplicados y normalización de los nombres de usuarios y menciones

In [9]:
# --- Tweets ---
def normalize_user(u):
    if not isinstance(u, str):
        return None
    return u.strip().lower()

df_trafico_norm = df_trafico_rel.copy()

# Normalizar username, reply_to_user, retweet_user
df_trafico_norm["username"] = df_trafico_norm["username"].apply(normalize_user)
if "reply_to_user" in df_trafico_norm.columns:
    df_trafico_norm["reply_to_user"] = df_trafico_norm["reply_to_user"].apply(normalize_user)
if "retweet_user" in df_trafico_norm.columns:
    df_trafico_norm["retweet_user"] = df_trafico_norm["retweet_user"].apply(normalize_user)

# Normalizar lista de menciones
df_trafico_norm["mentions_extracted"] = df_trafico_norm["mentions_extracted"].apply(
    lambda lst: [normalize_user(u) for u in lst if normalize_user(u)] if isinstance(lst, list) else []
)

# Eliminar tweets duplicados por ID
df_trafico_norm = df_trafico_norm.drop_duplicates(subset=["tweet_id"]).reset_index(drop=True)

print("Tweets tras normalización:", df_trafico_norm.shape)

# --- Aristas ---
edges_trafico_norm = edges_trafico.copy()
edges_trafico_norm["src"] = edges_trafico_norm["src"].apply(normalize_user)
edges_trafico_norm["dst"] = edges_trafico_norm["dst"].apply(normalize_user)

# Eliminar duplicados de aristas
edges_trafico_norm = edges_trafico_norm.drop_duplicates(subset=["src","dst","type","tweet_id"]).reset_index(drop=True)

print("Aristas tras normalización:", edges_trafico_norm.shape)

# Vista previa
print(edges_trafico_norm.head(10))


Tweets tras normalización: (5596, 26)
Aristas tras normalización: (14931, 5)
               src              dst     type             tweet_id  \
0     monymmorales     cc_guatemala  mention  1834029142565658846   
1  animaldgalaccia   pncdeguatemala  mention  1834039491826180424   
2  animaldgalaccia      mingobguate  mention  1834039491826180424   
3  animaldgalaccia   fjimenezmingob  mention  1834039491826180424   
4  animaldgalaccia     diegoedeleon  mention  1834039491826180424   
5  animaldgalaccia   amilcarmontejo  mention  1834039491826180424   
6  animaldgalaccia        traficogt  mention  1834039491826180424   
7  animaldgalaccia   pncdeguatemala    reply  1834039491826180424   
8   estaciondoblea   amilcarmontejo  mention  1833963729136091179   
9   estaciondoblea  aztecanoticiagt  mention  1833963729136091179   

                       date  
0 2024-09-12 00:39:56+00:00  
1 2024-09-12 01:21:04+00:00  
2 2024-09-12 01:21:04+00:00  
3 2024-09-12 01:21:04+00:00  
4 2024-09-12 

#### 3.4) Estructura para el análisis de redes

In [11]:
# 1) Asegurar tipos y quedarnos solo con interacciones válidas
valid_types = {"mention", "reply", "retweet"}
E = edges_trafico_norm.copy()
E = E[E["type"].isin(valid_types)].dropna(subset=["src","dst"]).reset_index(drop=True)
E["src"] = E["src"].astype(str).str.strip().str.lower()
E["dst"] = E["dst"].astype(str).str.strip().str.lower()

# 2) Ponderación (opcional). Puedes ajustar estos pesos si lo pide tu rúbrica
type_weights = {"mention": 1.0, "reply": 2.0, "retweet": 3.0}
E["w_type"] = E["type"].map(type_weights).fillna(1.0)

# 3) Agregar múltiples interacciones entre el mismo par (src→dst)
#    - w_count: número de interacciones
#    - w_sum:   suma de pesos por tipo
Agg = (
    E.groupby(["src","dst"], as_index=False)
      .agg(w_count=("type","size"), w_sum=("w_type","sum"))
)

# 4) Índices de nodos (user → idx) y vectores i,j para matriz dispersa
nodes = pd.Index(pd.unique(pd.concat([Agg["src"], Agg["dst"]], ignore_index=True)))
node_to_idx = {u: i for i, u in enumerate(nodes)}

row = Agg["src"].map(node_to_idx).to_numpy()
col = Agg["dst"].map(node_to_idx).to_numpy()

# 5) Construir matrices dispersas (COO→CSR):
#    - adj_count: peso = #interacciones
#    - adj_weight: peso = suma de pesos por tipo
adj_count  = coo_matrix((Agg["w_count"].to_numpy(), (row, col)), shape=(len(nodes), len(nodes))).tocsr()
adj_weight = coo_matrix((Agg["w_sum"].to_numpy(),   (row, col)), shape=(len(nodes), len(nodes))).tocsr()

# 6) Resultado principal para análisis de redes
#    - Edge list final: Agg (src, dst, w_count, w_sum)
#    - Matriz de adyacencia (CSR): adj_count / adj_weight
#    - Mapeo de nodos: nodes (índice→usuario) y node_to_idx (usuario→índice)

print(f"Nodos: {len(nodes)}")
print(f"Aristas dirigidas (colapsadas): {Agg.shape[0]}")
print("Ejemplo de edges:")
print(Agg.head(10))

# (Opcional) funciones auxiliares prácticas
def neighbors_out(user, topk=10, use_weights="count"):
    """Vecinos salientes y pesos desde 'user'."""
    if user not in node_to_idx:
        return pd.DataFrame(columns=["dst","weight"])
    i = node_to_idx[user]
    mat = adj_count if use_weights=="count" else adj_weight
    idxs = mat[i].indices
    vals = mat[i].data
    dst_users = nodes[idxs]
    out = pd.DataFrame({"dst": dst_users, "weight": vals}).sort_values("weight", ascending=False)
    return out.head(topk).reset_index(drop=True)

def neighbors_in(user, topk=10, use_weights="count"):
    """Vecinos entrantes y pesos hacia 'user'."""
    if user not in node_to_idx:
        return pd.DataFrame(columns=["src","weight"])
    j = node_to_idx[user]
    mat = adj_count if use_weights=="count" else adj_weight
    # columna j (entrantes) -> usamos la transpuesta
    col_vec = mat[:, j].tocoo()
    src_users = nodes[col_vec.row]
    out = pd.DataFrame({"src": src_users, "weight": col_vec.data}).sort_values("weight", ascending=False)
    return out.head(topk).reset_index(drop=True)

# Ejemplos de uso:
# neighbors_out("traficogt", topk=5, use_weights="count")
# neighbors_in ("traficogt", topk=5, use_weights="weight")


Nodos: 2720
Aristas dirigidas (colapsadas): 7338
Ejemplo de edges:
             src             dst  w_count  w_sum
0         01lu88  barevalodeleon        4    5.0
1         01lu88  emisorasunidas        3    3.0
2         01lu88          mcfrsa        2    3.0
3         01lu88       traficogt        3    3.0
4         0s_dev       traficogt        2    3.0
5   1056_antonio   manfredoguate        2    3.0
6   1056_antonio       traficogt        1    1.0
7         1601jr       traficogt        2    3.0
8  17147128204ca  barevalodeleon        2    3.0
9  17147128204ca           pdhgt        1    1.0


Se construyó la estructura de grafo dirigido que resume las interacciones entre usuarios de la cuenta @traficogt. El resultado final muestra una red con 2,720 nodos únicos (usuarios que participan) y 7,338 aristas dirigidas colapsadas, lo que significa que se agruparon todas las menciones, respuestas y retuits entre cada par de usuarios para evitar repeticiones. Cada arista registra no solo cuántas veces ocurrió la interacción (w_count), sino también un peso acumulado (w_sum) que diferencia la intensidad según el tipo de relación (mención=1, respuesta=2, retuit=3). De esta forma, la red queda representada en un formato eficiente que permite identificar con claridad quién interactúa con quién y con qué fuerza, preparando la base para calcular métricas de centralidad y detectar comunidades en el análisis posterior.